# Gateway CLMM Router Example

This notebook demonstrates how to use the Gateway CLMM router for concentrated liquidity market maker operations.

The Gateway CLMM router provides functionality for:
- Getting pool information
- Opening new CLMM positions with price ranges
- Adding/removing liquidity from positions
- Collecting fees from positions
- Tracking position history and performance

Supported protocols: Meteora, Raydium CLMM, Uniswap V3

## Setup

Initialize the Hummingbot API client.

In [ ]:
from hummingbot_api_client import HummingbotAPIClient
from decimal import Decimal

client = HummingbotAPIClient(base_url='http://localhost:8000')
await client.init()

## Get Pool Information

Get detailed information about a CLMM pool before providing liquidity.

In [ ]:
# Get pool information for a Meteora DLMM pool
pool_address = '2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3'

pool_info = await client.gateway_clmm.get_pool_info(
    connector='meteora',
    network='solana-mainnet-beta',
    pool_address=pool_address
)

print(f"Pool info: {pool_info}")

## Browse Available Pools

Get a list of available CLMM pools to find trading opportunities.

In [ ]:
# Get list of available Meteora pools
pools = await client.gateway_clmm.get_pools(
    connector='meteora',
    search_term='SOL',  # Search for SOL pairs
    limit=10,
    sort_key='volume',  # Sort by 24h volume
    order_by='desc'
)

print(f"Found {pools['total']} pools (showing {len(pools['pools'])} results)\n")

for pool in pools['pools']:
    print(f"Pool: {pool['trading_pair']}")
    print(f"  Address: {pool['address'][:16]}...")
    print(f"  Current Price: ${pool['current_price']}")
    print(f"  Liquidity: ${pool['liquidity']}")
    if pool.get('volume_24h'):
        print(f"  24h Volume: ${pool['volume_24h']}")
    if pool.get('apr'):
        print(f"  APR: {pool['apr']}%")
    print(f"  Verified: {pool['is_verified']}")
    print()

## Open a New CLMM Position

Open a new concentrated liquidity position with a specified price range.

**⚠️ WARNING:** This will execute a real transaction on-chain and costs gas fees!

In [ ]:
# Open a new position (uncomment to run)
result = await client.gateway_clmm.open_position(
    connector='meteora',
    network='solana-mainnet-beta',
    pool_address=pool_address,
    lower_price=Decimal('150'),  # Lower price bound
    upper_price=Decimal('250'),  # Upper price bound
    base_token_amount=Decimal('0.01'),  # Amount of base token
    quote_token_amount=Decimal('2'),    # Amount of quote token
    slippage_pct=Decimal('1.0'),
    extra_params={"strategyType": 2}  # Meteora strategy type
)

print(f"Position opened!")
print(f"Transaction Hash: {result['transaction_hash']}")
print(f"Position Address: {result['position_address']}")
print(f"Trading Pair: {result['trading_pair']}")
print(f"Price Range: {result['lower_price']} - {result['upper_price']}")

## View Positions Owned

Get all CLMM positions you own for a specific pool.

In [ ]:
# Get all positions for a pool
positions = await client.gateway_clmm.get_positions_owned(
    connector='meteora',
    network='solana-mainnet-beta',
    pool_address=pool_address
)

print(f"Found {len(positions)} positions:\n")
for pos in positions:
    print(f"Position: {pos['position_address'][:16]}...")
    print(f"  Trading Pair: {pos['trading_pair']}")
    print(f"  Price Range: {pos['lower_price']} - {pos['upper_price']}")
    print(f"  Current Price: {pos['current_price']}")
    print(f"  In Range: {pos['in_range']}")
    print(f"  Base Amount: {pos['base_token_amount']}")
    print(f"  Quote Amount: {pos['quote_token_amount']}")
    if pos.get('base_fee_amount'):
        print(f"  Pending Fees: {pos['base_fee_amount']} / {pos['quote_fee_amount']}")
    print()

## Collect Fees from Position

Collect accumulated trading fees from your position.

In [ ]:
# Collect fees (uncomment to run)
# position_address = 'YOUR_POSITION_ADDRESS'
#
# result = await client.gateway_clmm.collect_fees(
#     connector='meteora',
#     network='solana-mainnet-beta',
#     position_address=position_address
# )
#
# print(f"Fees collected!")
# print(f"Transaction: {result['transaction_hash']}")
# print(f"Base Token Fees: {result['base_fee_collected']}")
# print(f"Quote Token Fees: {result['quote_fee_collected']}")

## Close Position Completely

Close a position by removing all liquidity.

In [ ]:
# Close position (uncomment to run)
# position_address = 'YOUR_POSITION_ADDRESS'
#
# result = await client.gateway_clmm.close_position(
#     connector='meteora',
#     network='solana-mainnet-beta',
#     position_address=position_address
# )
#
# print(f"Position closed!")
# print(f"Transaction: {result['transaction_hash']}")

## Search Position History

Search through your CLMM positions with various filters.

In [ ]:
# Search all open positions on Solana
positions = await client.gateway_clmm.search_positions(
    network='solana-mainnet-beta',
    connector='meteora',
    status='OPEN',
    limit=10,
    refresh=True  # Refresh data from Gateway
)

print(f"Found {len(positions['data'])} open positions\n")
for pos in positions['data']:
    print(f"Position: {pos['position_address'][:16]}...")
    print(f"  Pool: {pos['trading_pair']}")
    print(f"  Range: {pos['lower_price']} - {pos['upper_price']}")
    print(f"  In Range: {pos['in_range']}")
    print(f"  Base: {pos['base_token_amount']}")
    print(f"  Quote: {pos['quote_token_amount']}")
    print()

In [ ]:
# Search positions by trading pair
pair_positions = await client.gateway_clmm.search_positions(
    network='solana-mainnet-beta',
    trading_pair='SOL-USDC',
    limit=5
)

print(f"SOL-USDC positions: {len(pair_positions['data'])}")

In [ ]:
# Search specific positions by address
# specific_positions = await client.gateway_clmm.search_positions(
#     position_addresses=['ADDRESS_1', 'ADDRESS_2'],
#     refresh=True
# )
# 
# for pos in specific_positions['data']:
#     print(f"Position {pos['position_address']}")
#     print(f"  Status: {pos['status']}")
#     print(f"  In Range: {pos['in_range']}")

## View Position Events

Get the event history for a specific position.

In [ ]:
# Get all events for a position
# position_address = 'YOUR_POSITION_ADDRESS'
# 
# events = await client.gateway_clmm.get_position_events(
#     position_address=position_address,
#     limit=50
# )
# 
# print(f"Position Event History:\n")
# for event in events['data']:
#     print(f"{event['event_type']} - {event['created_at']}")
#     print(f"  TX: {event['transaction_hash'][:16]}...")
#     print(f"  Status: {event['status']}")
#     
#     if event['event_type'] in ['OPEN', 'ADD_LIQUIDITY']:
#         print(f"  Added: {event.get('base_token_amount')} / {event.get('quote_token_amount')}")
#     elif event['event_type'] == 'REMOVE_LIQUIDITY':
#         print(f"  Removed: {event.get('percentage')}%")
#     elif event['event_type'] == 'COLLECT_FEES':
#         print(f"  Collected: {event.get('base_fee_collected')} / {event.get('quote_fee_collected')}")
#     print()

In [ ]:
# Get only fee collection events
# fee_events = await client.gateway_clmm.get_position_events(
#     position_address=position_address,
#     event_type='COLLECT_FEES',
#     limit=10
# )
# 
# print(f"Fee Collections:\n")
# total_base_fees = Decimal('0')
# total_quote_fees = Decimal('0')
# 
# for event in fee_events['data']:
#     base_fee = Decimal(str(event.get('base_fee_collected', 0)))
#     quote_fee = Decimal(str(event.get('quote_fee_collected', 0)))
#     total_base_fees += base_fee
#     total_quote_fees += quote_fee
#     print(f"  {event['created_at']}: {base_fee} / {quote_fee}")
# 
# print(f"\nTotal Fees Collected:")
# print(f"  Base: {total_base_fees}")
# print(f"  Quote: {total_quote_fees}")

## Complete Workflow Example

A complete workflow for managing a CLMM position.

In [ ]:
# 1. Get pool info and check current price
print("Step 1: Checking pool information...")
pool_info = await client.gateway_clmm.get_pool_info(
    connector='meteora',
    network='solana-mainnet-beta',
    pool_address=pool_address
)

current_price = Decimal(str(pool_info['current_price']))
print(f"Current Price: {current_price}")

# 2. Calculate price range (±10% from current)
print("\nStep 2: Calculating price range...")
lower_price = current_price * Decimal('0.9')  # -10%
upper_price = current_price * Decimal('1.1')  # +10%
print(f"Price Range: {lower_price} - {upper_price}")

# 3. Open position (uncomment to execute)
print("\nStep 3: Ready to open position (uncomment to execute)")
# result = await client.gateway_clmm.open_position(
#     connector='meteora',
#     network='solana-mainnet-beta',
#     pool_address=pool_address,
#     lower_price=lower_price,
#     upper_price=upper_price,
#     base_token_amount=Decimal('0.1'),
#     quote_token_amount=Decimal('10'),
#     slippage_pct=Decimal('1.0'),
#     extra_params={"strategyType": 0}
# )
# 
# position_address = result['position_address']
# print(f"\n✅ Position opened!")
# print(f"Position Address: {position_address}")
# print(f"Transaction: {result['transaction_hash']}")

# 4. Monitor position
print("\nStep 4: Monitor position status with search_positions()")

# 5. Collect fees periodically
print("Step 5: Collect fees periodically with collect_fees()")

# 6. Adjust or close when needed
print("Step 6: Adjust liquidity or close position when needed")

## Monitoring Positions: In Range vs Out of Range

Check which positions are currently earning fees (in range) vs out of range.

In [ ]:
# Get all positions and check range status
positions = await client.gateway_clmm.search_positions(
    network='solana-mainnet-beta',
    connector='meteora',
    status='OPEN',
    refresh=True,  # Get latest data
    limit=50
)

in_range_positions = [p for p in positions['data'] if p.get('in_range') == 'IN_RANGE']
out_of_range_positions = [p for p in positions['data'] if p.get('in_range') == 'OUT_OF_RANGE']

print(f"Position Summary:")
print(f"  Total Open: {len(positions['data'])}")
print(f"  In Range (earning fees): {len(in_range_positions)}")
print(f"  Out of Range: {len(out_of_range_positions)}")

if out_of_range_positions:
    print(f"\n⚠️ Out of Range Positions:")
    for pos in out_of_range_positions:
        print(f"  {pos['trading_pair']} - {pos['position_address'][:16]}...")
        print(f"    Range: {pos['lower_price']} - {pos['upper_price']}")
        print(f"    Current: {pos.get('current_price', 'N/A')}")

## Tips and Best Practices

1. **Check pool info first** - Understand current price, liquidity, and fees before opening positions
2. **Choose appropriate price ranges** - Narrower ranges = higher fees but more rebalancing needed
3. **Monitor in_range status** - Positions only earn fees when price is within range
4. **Collect fees regularly** - Compound your earnings by collecting and reinvesting fees
5. **Use refresh parameter** - Set `refresh=True` in search to get real-time position data
6. **Track position events** - Review event history to analyze position performance
7. **Set appropriate slippage** - 1-2% is typical for most CLMM operations
8. **Consider gas costs** - Each operation costs gas, factor this into profitability
9. **Rebalance strategically** - Close and reopen positions when price moves significantly
10. **Start small** - Test with small amounts before committing significant capital

## Understanding Meteora DLMM Strategy Types

When opening Meteora positions, you can specify a strategy type:

- **Strategy 0 (Spot)**: Standard liquidity provision across price range
- **Strategy 1 (Curve)**: Concentrated around current price
- **Strategy 2 (Bid-Ask)**: Separate buy and sell ranges

Example:
```python
extra_params={"strategyType": 0}  # Spot strategy
```

## Cleanup

Close the client when done.

In [ ]:
await client.close()